In [1]:
import subprocess
import pandas as pd
import numpy as np

In [ ]:
bases=["C","T","G","A"]
for bas in bases: 
    inp2="/mnt/mnemo6/lukas/anaconda3/envs/sar3/bin/tRNAscan-SE -G /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_all_viral_sequences_v2.fasta -o /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_v2.txt -f /mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_secondary_struc.txt"
    subprocess.run(inp2, shell=True)

In [31]:
#perhaps try only unique sequences
bases=["C","T","G","A"]
run=["first","second","third","fourth"]
wholedic={}
wholedic["trna"]={}
wholedic["ribo"]={}
#save the unique sequences to check later for unique sequences (excluding others from the C search)
wholedic["dfs"]={}
for nr in run:
    wholedic["trna"][nr]={}
    wholedic["ribo"][nr]={}
    wholedic["dfs"][nr]={}
    for bas in bases: 
        rnarobo_=open("/mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/"+nr+"/"+bas+"/comb_rnarobo_result_precise.txt")
        rnarobo=rnarobo_.readlines()
        rnarobo_.close()
        if nr=="fourth":
            trna_=open("/mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/fourth/"+bas+"/combined_trna_scan_v2.txt")
            trna=trna_.readlines()
            trna_.close()

        else:
            trna_=open("/mnt/mnemo6/lukas/Python/ribozymes/genome/combined_v3/falsepositive/first/"+bas+"/combined_trna_scan_v2.txt")
            trna=trna_.readlines()
            trna_.close()

        dicti={}
        for line in trna[3:]:
            #create a dictionary with seq name as key and the starting position as value
                line=line.strip().split()
                if line[0] in dicti:
                    dicti[line[0]].append(int(line[3]))
                else:
                    dicti[line[0]]=[int(line[3])]




        dicti_r={}
        ribis=[]
        for line in rnarobo[11:-6]:
            if line[0]!="|":
                line=line.replace(".","_")
                line=line.strip().split()
                if line[2] in dicti_r:
                    dicti_r[line[2]].append(int(line[0]))
                else:
                    dicti_r[line[2]]=[int(line[0])]
            #attach sequence if unique
            if line[0]=="|":
                lin_stripped="".join(line.strip().split("|"))
                if lin_stripped not in ribis:
                    ribis.append(lin_stripped)
            
                
        
        wholedic["trna"][nr][bas]=dicti
        wholedic["ribo"][nr][bas]=dicti_r
        matching=[]
        for key in dicti_r:
            if key in dicti:
                for value in dicti_r[key]:
                    for value2 in dicti[key]:
                        if value-value2 in range(-5,6):
                            matching.append([key,value,value2])
                            
        newm=[]
        for mat in matching:
            if mat not in newm:
                newm.append(mat)
                
                
        #check for the dataframe:
        
        
        lets_do_this=pd.DataFrame(columns=["ID","trna_nr","trna_start","trna_end","trna_type","codon","ins_start","ins_end","trna_confidence","psuedo","rnarobo_start","rnarobo_end","rnarobo_seq"])
        check=0
        duplicates=[]
        for numbr in range(len(newm)):
            match=newm[numbr]
            lin_df=""
            for line in trna[3:]:
            #create a dictionary with seq name as key and the starting position as value
                line=line.strip().split()

                if match[0]==line[0]:
                    if len(lin_df.split())==0:
                        if str(match[2])==line[3]:
                            #print(line)
                            lin_df+=" ".join(line)+" "
                            if lin_df[-2]!="o":
                                lin_df+="NA "
                    else: 
                        duplicates.append(line)
            for line in rnarobo[11:-6]:
                line=line.replace(".","_")
                line=line.strip().split()
                if len(lin_df.split())<13:

                    if check==1:
                        #if len(lin_df.split())<13:
                            lin_df+=str(line[0])
                            check=0
                        #else:
                                #duplicates.append(line)

                        #check=0
                    if str(match[1])==line[0]:
                        #if len(lin_df.split())<13:
                            lin_df+=" ".join(line[:2])
                            lin_df+=" "
                            check=1
                    #else:
                        #duplicates.append(line)
                        #check=0
            lin_f=lin_df.split()
            lets_do_this.loc[numbr]=np.array(lin_f)
            #lets_do_this.iloc[nr,:8]=lin_df[0]
            #lets_do_this.iloc[nr,8:-1]=lin_df[1]
            #lets_do_this.iloc[nr,-1]=lin_df[2]
        wholedic["dfs"][nr][bas]=ribis
        
        #number of ribozymes found directly together with trna
        #ln_m=0
       # for x in newm:
        #    ln_m+=len(dicti_r[x])
        #number ob ribozymes found(total)
        ln=0
        for x in dicti_r:
            ln+=len(dicti_r[x])
        print(nr, bas,len(ribis),len(lets_do_this["rnarobo_seq"].unique()))

first C 840 105
first T 424 49
first G 301 2
first A 284 12
second C 508 97
second T 160 38
second G 48 2
second A 64 2
third C 193 76
third T 42 30
third G 3 1
third A 5 1
fourth C 302 127
fourth T 53 34
fourth G 8 5
fourth A 11 3


In [32]:
#check for the duplicate hits (e.g. false positives which where found in both CNRA* as well as TNRA*)
for nr in run:
    for bas in bases[:]:
        dictiT_r=wholedic["dfs"][nr][bas]
        dicti_r=wholedic["dfs"][nr]["C"]
        newm=[]
        #now lets see which of the unique ribozymes of the dataframe are also found in the C one
        for y in set(dictiT_r):
            if y in set(dicti_r):
                newm.append(y)
        print(nr,bas,len(newm))

first C 840
first T 1
first G 0
first A 0
second C 508
second T 1
second G 0
second A 0
third C 193
third T 0
third G 0
third A 0
fourth C 302
fourth T 41
fourth G 6
fourth A 9


In [24]:
#check for the duplicate hits (e.g. false positives which where found in both CNRA* as well as TNRA*)
for nr in run:
    for bas in bases[1:]:
        dictiT_r=wholedic["ribo"][nr][bas]
        dicti_r=wholedic["ribo"][nr]["C"]
        matching_both=[]
        for key in dictiT_r:
            if key in dicti_r:
                for value in dictiT_r[key]:
                    for value2 in dicti_r[key]:
                        if value-value2 in range(-1,1):
                            matching_both.append([key,value,value2])
        newm=[]
        for mat in matching_both:
            if mat not in newm:
                newm.append(mat)
        print(nr,bas,len(newm))

first T 19
first G 8
first A 134
second T 1
second G 0
second A 33
third T 0
third G 0
third A 0
fourth T 231
fourth G 89
fourth A 33
